In [1]:
# TODO Attribution from http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/
import nltk
import csv
import pickle

In [2]:
# This is specific to the dataset
negative_label = 'negative'
neutral_label = 'neutral'
positive_label = 'positive'
irrelevant_label = 'irrelevant'
with open('./../../data/external/sanders.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Sentiment and tweet are switched in the dataset
    tweets = [(tweet, sentiment) for (topic, sentiment, id, tweet) in iterator]


In [3]:
# This is also specific to the dataset
def preprocess(tweets):
    tokenizer = nltk.tokenize.TweetTokenizer()
    preprocessed_tweets = []
    for (text, sentiment) in tweets:
        words_filtered = tokenizer.tokenize(text)
        preprocessed_tweets.append((words_filtered, sentiment))
    return preprocessed_tweets

In [4]:
# Preprocess tweets
preprocessed_tweets = preprocess(tweets)
preprocessed_tweets[0]

(['"',
  '@Twitter',
  'CEO',
  'points',
  'to',
  '@Apple',
  'as',
  "'",
  'corporate',
  'mentor',
  "'",
  'as',
  '@iOS',
  'signups',
  'triple',
  'http://t.co/GCY8iphN',
  '"'],
 'positive')

In [5]:
"There are %d tweets in this dataset" % len(preprocessed_tweets)

'There are 4345 tweets in this dataset'

In [6]:
# Split between training and test set
# There are 4345 tweets in the dataset
train_tweets = preprocessed_tweets[:3500]
test_tweets = preprocessed_tweets[3500:]

In [7]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
      all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features


In [8]:
word_features = get_word_features(get_words_in_tweets(train_tweets))

In [9]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [10]:
training_set = nltk.classify.apply_features(extract_features, train_tweets)

In [11]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [12]:
tweet = 'Larry is not my friend'
print(classifier.classify(extract_features(tweet.split())))

neutral


In [13]:
labeled_correctly = 0
for tweet in test_tweets:
    if classifier.classify(extract_features(tweet[0])) == tweet[1]:
        labeled_correctly+=1
print("%d/%d" % (labeled_correctly,len(test_tweets)))        

693/845


In [15]:
# Save the classifier if it's any good
save_classifier = open("./../../models/naive_bayes/nltk_naive_bayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()